In [1]:
# 加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# 定义超参数
batch_size = 30 # 每批处理的数据
device = torch.device( "cpu")
epochs = 10 # 训练数据集的轮次

In [3]:
# 数据预处理   
pipeline = transforms.Compose([
    transforms.ToTensor(), # 将图片转换为tensor
    transforms.Normalize((0.1307,), (0.3081,)) # 正则化，降低模型复杂度，参数官网的  
])


In [4]:
# 下载，加载数据集
from torch.utils.data import DataLoader

train_set = datasets.MNIST("data", train=True, download=True, transform=pipeline)

test_set = datasets.MNIST("data", train=False, download=True, transform=pipeline) #下载数据集

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [5]:
## 显示MNIST中的图片
with open("./data/MNIST/raw/t10k-images-idx3-ubyte", "rb") as f:
    file = f.read()

In [6]:
image1 = [int(str(item).encode('ascii'), 16) for item in file[16 : 16+784]]

In [7]:
import cv2
import numpy as np

image1_np = np.array(image1, dtype=np.uint8).reshape(28, 28, 1)

print(image1_np.shape)

(28, 28, 1)


C:\Users\86182\AppData\Local\Temp\ipykernel_13788\3272523839.py:4: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 389 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  image1_np = np.array(image1, dtype=np.uint8).reshape(28, 28, 1)
C:\Users\86182\AppData\Local\Temp\ipykernel_13788\3272523839.py:4: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 345 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  image1_np = np.array(image1, dtype=np.uint8).reshape(28, 28, 1)
C:\Users\86182\AppData\Local\Temp\ipykernel_13788\3272523839.py:4: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays

In [8]:
cv2.imwrite("digit.jpg", image1_np)

True

In [29]:
# 构建网络模型
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)  # 灰度图片的通道， 输出通道， 卷积核
        self.conv2 = nn.Conv2d(10, 20, 3) # 输入通道， 输出通道， 卷积核
        self.fc1 = nn.Linear(20*10*10, 500) # 输入通道， 输出通道
        self.fc2 = nn.Linear(500, 10) #输入通道， 输出通道
        
    def forward(self, x):
        input_size = x.size(0) # batch_size
        x = self.conv1(x)  # 输入：batch*1*28*28, 输出：batch*10*24*24 (28 - 5 + 1)
        x = F.relu(x)   # 激活函数 保持shape不变，输出batch*10*24*24
        x = F.max_pool2d(x, 2, 2) # 池化层，图片压缩 输入：batch*10*24*24 输出：batch*10*12*12

        x = self.conv2(x)
        x = F.relu(x)

        x = x.view(input_size, -1) # 拉平，自动计算维度20*10*10 = 2000

        x = self.fc1(x)  # 输入：batch*2000 输出batch*500
        x = F.relu(x) # 保持shape不变

        x = self.fc2(x) # 输入batch*500 输出batch*10

        output = F.log_softmax(x, dim=1) # 计算分类后，每个数字的概率值 按行计算

        return output # 返回概率值

In [30]:
# 定义优化器
model = Digit().to(device)

optimizer = optim.Adam(model.parameters())

In [31]:
# 定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    # 模型训练
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        # 部署到device上去
        data, target = data.to(device), target.to(device)
        #梯度初始化0
        optimizer.zero_grad()
        # 训练后的结果
        output = model(data)
        # 计算损失
        loss = F.cross_entropy(output, target)    # 交叉熵
        # 反向传播
        loss.backward()
        # 参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print("Train Epoch : {} \t Loss : {:.6f}".format(epoch, loss.item()))        

In [32]:
# 定义测试方法
def test_model(model, device, test_loader):
    # 模型验证
    model.eval()
    # 正确率
    correct = 0.0
    #测试损失
    test_loss = 0.0
    with torch.no_grad(): #不计算梯度，也不反向传播
        for data, target in test_loader:
            # 部署到device
            data, target = data.to(device), target.to(device)
            # 测试数据
            output = model(data)
            # 计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            # 找到概率值最大的下标
            pred = output.max(1, keepdim=True)[1] # 值 索引
            # 累计正确率
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("Test —— Average loss : {:.4f}, Accuracy : {:.3f}\n".format(
            test_loss, 100.0 * correct / len(test_loader.dataset)))
        

In [33]:
# 调用方法
for epoch in range(1, epochs + 1):
    train_model(model, device, train_loader, optimizer, epoch)
    test_model(model, device, test_loader)

Train Epoch : 1 	 Loss : 2.308095
Test —— Average loss : 0.0015, Accuracy : 98.450

Train Epoch : 2 	 Loss : 0.024467
Test —— Average loss : 0.0014, Accuracy : 98.550

Train Epoch : 3 	 Loss : 0.002788
Test —— Average loss : 0.0014, Accuracy : 98.620

Train Epoch : 4 	 Loss : 0.010644
Test —— Average loss : 0.0012, Accuracy : 98.910

Train Epoch : 5 	 Loss : 0.012657
Test —— Average loss : 0.0015, Accuracy : 98.760

Train Epoch : 6 	 Loss : 0.000001
Test —— Average loss : 0.0015, Accuracy : 98.860

Train Epoch : 7 	 Loss : 0.070084
Test —— Average loss : 0.0015, Accuracy : 98.990

Train Epoch : 8 	 Loss : 0.001633
Test —— Average loss : 0.0026, Accuracy : 98.650

Train Epoch : 9 	 Loss : 0.000754
Test —— Average loss : 0.0014, Accuracy : 99.090

Train Epoch : 10 	 Loss : 0.000667
Test —— Average loss : 0.0017, Accuracy : 98.960

